In [1]:
# ==============================================================================
# 文件名: 03_final_training_xgb.py
# 描述: 针对 CDFT + RDKT 的最终训练脚本 (生产环境版)
# 功能: 1. 支持指挥官模式的种子注入 (优先命令行参数，其次环境变量)
#       2. 处理 Train+Val 合并训练
#       3. [新增] MD5 数据指纹校验
# ==============================================================================

import os
import json
import pandas as pd
import numpy as np
import hashlib
import argparse
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error

def calculate_file_hash(filepath):
    """计算文件的 MD5 哈希值，用于验证数据是否真的发生了变化"""
    if not os.path.exists(filepath):
        return "File_Not_Found"
    with open(filepath, "rb") as f:
        file_hash = hashlib.md5(f.read()).hexdigest()
    return file_hash[:8] # 只取前8位即可

def run_final_training():
    # ---------------------------------------------------------
    # 1. 动态环境识别 (命令行参数 > 环境变量 > 默认)
    # ---------------------------------------------------------
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', type=int, default=None, help='接收外部注入的随机种子')
    args, unknown = parser.parse_known_args()

    env_seed = os.environ.get('STABILITY_SEED')

    if args.seed is not None:
        current_seed = args.seed
        print(f"🚀 [稳定性测试] 接收到命令行种子: {current_seed}")
    elif env_seed is not None:
        current_seed = int(env_seed)
        print(f"🚀 [稳定性测试] 检测到环境变量种子: {current_seed}")
    else:
        current_seed = 42
        print(f"ℹ️ [标准模式] 未检测到外部种子，使用默认: {current_seed}")

    # ---------------------------------------------------------
    # 2. 数据加载与路径处理
    # ---------------------------------------------------------
    # 自动识别路径：支持在项目根目录或 notebooks 目录下运行
    data_dir = 'data/processed'
    if not os.path.exists(data_dir) and os.path.exists('../data/processed'):
        data_dir = '../data/processed'

    # [诊断核心] 打印数据指纹
    train_path = f'{data_dir}/train_data.csv'
    test_path = f'{data_dir}/test_data.csv'
    train_hash = calculate_file_hash(train_path)
    test_hash = calculate_file_hash(test_path)

    print(f"🔍 [数据法医] Train Hash: {train_hash} | Test Hash: {test_hash}")

    try:
        df_train = pd.read_csv(train_path)
        df_val = pd.read_csv(f'{data_dir}/val_data.csv')
        df_test = pd.read_csv(test_path)
    except Exception as e:
        print(f"❌ 数据读取失败: {e}")
        return

    target_col = 'Target_Log1o2'
    # 排除非特征列 (SMILES 和 Target)
    drop_cols = [target_col, 'SMILES']
    if 'Unnamed: 0' in df_train.columns: drop_cols.append('Unnamed: 0')

    # 合并 Train + Val 进行全量训练 (最大化利用 CDFT/RDKT 特征信息)
    X_train_val = pd.concat([df_train, df_val], ignore_index=True).drop(columns=drop_cols)
    y_train_val = pd.concat([df_train[target_col], df_val[target_col]], ignore_index=True)

    X_test = df_test.drop(columns=drop_cols)
    y_test = df_test[target_col]

    # ---------------------------------------------------------
    # 3. 参数配置 (模型“随机性”控制)
    # ---------------------------------------------------------
    # 读取 Step 02 生成的最佳超参数
    res_model_dir = 'results/models'
    os.makedirs(res_model_dir, exist_ok=True)
    param_path = f'{res_model_dir}/best_params_xgb.json'

    best_params = {}
    if os.path.exists(param_path):
        with open(param_path, 'r') as f:
            raw_params = json.load(f)
        # 清洗参数名称 (去除 pipeline 前缀)
        best_params = {k.replace('xgb__', ''): v for k, v in raw_params.items()}

    # 核心模型参数
    params = {
        'objective': 'reg:squarederror',
        'random_state': current_seed,
        'n_jobs': -1,
        'tree_method': 'hist', # 推荐 CDFT 项目使用
        'n_estimators': 500,
        'learning_rate': 0.05
    }
    params.update(best_params)

    # ---------------------------------------------------------
    # 4. 训练与最终评估
    # ---------------------------------------------------------
    # print(f"⏳ 正在拟合最终模型 (Seed={current_seed})...")
    model = XGBRegressor(**params)
    model.fit(X_train_val, y_train_val)

    # 在独立的测试集上验证 (外部验证)
    y_pred_test = model.predict(X_test)
    r2 = r2_score(y_test, y_pred_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

    print('\n' + '='*40)
    print(f"📊 测试集评估结果 (Seed={current_seed})")
    print(f"   R2 Score: {r2:.4f}")
    print(f"   RMSE    : {rmse:.4f}")
    print('='*40)

    # ---------------------------------------------------------
    # 5. 结果持久化 (供 04_visualization 使用)
    # ---------------------------------------------------------
    res_pred_dir = 'results/predictions'
    os.makedirs(res_pred_dir, exist_ok=True)

    # 构造输出表格
    df_test_out = df_test.copy()
    df_test_out['Predicted'] = y_pred_test
    df_test_out['Residual'] = df_test_out[target_col] - y_pred_test

    # 保存两份：一份带模型后缀，一份通用名
    df_test_out.to_csv(f'{res_pred_dir}/final_predictions_xgb.csv', index=False)
    df_test_out.to_csv(f'{res_pred_dir}/final_predictions.csv', index=False)

    # 保存模型二进制
    model.save_model(f'{res_model_dir}/best_xgb_model.json')

    return r2

if __name__ == "__main__":
    run_final_training()

ℹ️ [标准模式] 未检测到外部种子，使用默认: 42
🔍 [数据法医] Train Hash: aedeab6e | Test Hash: 00369103



📊 测试集评估结果 (Seed=42)
   R2 Score: 0.6373
   RMSE    : 1.1147
